In [ ]:
import os
import pickle
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import * 

In [ ]:
from numpy.random import seed
seed(1)

tf.random.set_seed(2)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
with open('dataset_1000.pkl', 'rb') as infile:
    dataset_dict = pickle.load(infile)

In [ ]:
def create_CNN_GRU_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=128,
                                    kernel_size=5,
                                    activation='relu',
                                    padding='same'))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(50, return_sequences=True)))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(30,return_sequences=True)))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=1)))
    return model

def generate_params(model, name, X_train, y_train, X_valid, y_valid):    
    params = {
        'model': model,
        'batch_size': 256,
        'optimizer': tf.keras.optimizers.Adam(learning_rate=0.0005),
        'loss': tf.keras.losses.Huber(),
        'metrics': [tf.keras.metrics.MeanAbsoluteError(name='mae'), tf.keras.metrics.RootMeanSquaredError(name='rmse')],
        'callbacks': [tf.keras.callbacks.ModelCheckpoint(os.path.join('models', '{}.h5'.format(name)), monitor='val_loss', mode='min', save_best_only=True)],
        'epochs': 50,
        'X': X_train,
        'y': y_train,
        'X_valid': X_valid, 
        'y_valid': y_valid
    }
    return params

def create_and_train_model(params):
    model, batch_size, optimizer, loss, metrics, callbacks, epochs, X, y, X_valid, y_valid = params.values()
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid), callbacks=callbacks)
    return model

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
architecture = create_CNN_GRU_model()
X_train, y_train = np.expand_dims(dataset_dict['X_train'], 2), np.expand_dims(dataset_dict['y_train'], 2)
X_valid, y_valid = np.expand_dims(dataset_dict['X_valid'], 2), np.expand_dims(dataset_dict['y_valid'], 2)
params = generate_params(architecture, "CNN+GRU", X_train, y_train, X_valid, y_valid)
model = create_and_train_model(params)

In [ ]:
tf.keras.models.save_model(model, '/home/user/cnn_gru')